In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import logging

logging.basicConfig(level=logging.INFO)

In [3]:
import malaya
from tqdm import tqdm
from glob import glob
import json

/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

In [4]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/twitter/compiled-2022-06-08-twitter.tar
# !tar -xf compiled-2022-06-08-twitter.tar
# !rm compiled-2022-06-08-twitter.tar

In [5]:
files = sorted(glob('2022-06-08/*.json'))
files[:2]

['2022-06-08/2022-06-08-bahasa-0.json', '2022-06-08/2022-06-08-bahasa-1.json']

In [7]:
corrector = malaya.spell.probability()

INFO:huggingface:downloading frozen huseinzol05/v27-preprocessing/bm_1grams.json


In [8]:
nmt = malaya.translation.en_ms.transformer(model = 'base')
nmt_func = lambda x: nmt.greedy_decoder([x])[0]

INFO:frozen_graph:running home/ubuntu/.cache/huggingface/hub using device /device:CPU:0
2022-06-09 15:32:13.524105: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-09 15:32:13.527705: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-09 15:32:13.527720: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-06-09 15:32:13.527723: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel-desktop
2022-06-09 15:32:13.527762: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Not found: 

In [9]:
segmenter = malaya.segmentation.transformer(model = 'base')
segmenter_func = lambda x: segmenter.greedy_decoder([x])[0]

INFO:frozen_graph:running home/ubuntu/.cache/huggingface/hub using device /device:CPU:0


In [11]:
strings = []
for f in files[:2]:
    with open(f) as fopen:
        data = json.load(fopen)
    for d in data:
        if d['retweet_text_full'] == 'NULL':
            t = d['data_text']
        else:
            t = d['retweet_text_full']
        if len(t) > 2:
            strings.append(t)
    
len(strings)

200000

In [12]:
normalizer = malaya.normalize.normalizer(corrector)

In [13]:
normalizer.normalize(data[0]['data_text'],
                    normalize_entity = False,
                    normalize_year = False,
                    normalize_telephone = False)

{'normalize': 'Auto teringat adik sendiri', 'date': {}, 'money': {}}

In [14]:
normalizer.normalize(data[0]['data_text'],
                    normalize_entity = False,
                    normalize_year = False,
                    normalize_telephone = False,
                    translator = nmt_func,
                    segmenter = segmenter_func)

{'normalize': 'Auto teringat adik sendiri', 'date': {}, 'money': {}}

In [15]:
fast_text = malaya.language_detection.fasttext()

INFO:huggingface:downloading frozen huseinzol05/v34-language-detection/fasttext-malaya.ftz


In [16]:
%%time
fast_text.predict(strings[:1])[0]

CPU times: user 171 µs, sys: 104 µs, total: 275 µs
Wall time: 278 µs


'malay'

In [17]:
strings = list(set(strings))
len(strings)

106740

In [18]:
normalizer = malaya.normalize.normalizer()

In [ ]:
import re

results = []
for i in tqdm(range(len(strings))):
    if fast_text.predict([strings[i]])[0] not in ['malay', 'rojak']:
        continue
    try:
        r = normalizer.normalize(strings[i],
                normalize_entity = False,
                normalize_year = True,
                normalize_telephone = False,
                segmenter = segmenter_func)
        results.append((strings[i], re.sub(r'[ ]+', ' ', r['normalize']).strip()))
    except:
        pass

  1%|          | 615/106740 [00:30<1:14:43, 23.67it/s]

In [ ]:
len(results)

In [ ]:
with open('normalized.json', 'w') as fopen:
    json.dump(results, fopen)